In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from helper import get_openai_api_key
KEY = get_openai_api_key()

In [3]:
from openai import OpenAI

# Instantiate the client
client = OpenAI(
    api_key=KEY
)

In [4]:
#Define structure with Pydantic
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    email: Optional[str] = None

In [10]:
#Simulating a user post from a social media platform
completion = client.responses.parse(
    model="gpt-5.1",
    input=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Make up a user."},
    ],
    text_format=User,
)

In [17]:
user = completion.output_parsed
user

User(name='Jordan Lee', age=29, email='jordan.lee@example.com')

In [21]:
## Social media mention structure
from pydantic import BaseModel
from enum import Enum
from typing import List, Optional, Literal
from openai import OpenAI

class Mention(BaseModel):
    # The model chooses the product the mention is about,
    # as well as the social media post's sentiment
    product: Literal['app', 'website', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']

    # Model can choose to respond to the user
    needs_response: bool
    response: Optional[str]

    # If a support ticket needs to be opened, 
    # the model can write a description for the
    # developers
    support_ticket_description: Optional[str]

In [22]:
# Example mentions
mentions = [
    # About the app
    "@techcorp your app is amazing! The new design is perfect",
    # Website is down, negative sentiment + needs a fix
    "@techcorp website is down again, please fix!",
    # Nothing to respond to
    "hey @techcorp you're so evil"
]

In [ ]:
#Create a function to analyze the mention and provide structured output

def analyze_mention(
    mention: str, 
    personality: str = "friendly"
) -> Mention:
    completion = client.responses.parse(
        model="gpt-5.1",
        input=[
            {"role": "system", "content": f"""
                Extract structured information from 
                social media mentions about our products.

                Provide
                - The product mentioned (website, app, not applicable)
                - The mention sentiment (positive, negative, neutral)
                - Whether to respond (true/false). Don't respond to 
                  inflammatory messages or bait.
                - A customized response to send to the user if we need 
                  to respond.
                - An optional support ticket description to create.

                Your personality is {personality}.
            """},
            {"role": "user", "content": mention},
        ],
        text_format=Mention,
    )
    return completion.output_parsed

In [ ]:
#Example of positive mention
print("User post:", mentions[0])
processed_mention = analyze_mention(mentions[0])
processed_mention

User post: @techcorp your app is amazing! The new design is perfect


Mention(product='app', sentiment='positive', needs_response=True, response='Thanks so much for the love! We’re really glad you’re enjoying the new app design. If you have any ideas or features you’d like to see next, we’re all ears.', support_ticket_description=None)

In [ ]:
#Example of negative mention
print("User post:", mentions[1])
negative_mention = analyze_mention(mentions[1])
negative_mention


User post: @techcorp website is down again, please fix!


Mention(product='website', sentiment='negative', needs_response=True, response='Sorry you’re running into this! Our team is checking the website right now. If you don’t mind, could you share what you see on your screen (error message or code) and your location/device? That will help us get things back up faster.', support_ticket_description='User reports that the company website is down again. Investigate current uptime status, recent outages, and any recurring issues. Request any visible error messages, error codes, and details on user location and device/browser to help diagnose.')

In [28]:
mention_json_string = processed_mention.model_dump_json(indent=2)
print(mention_json_string)

{
  "product": "app",
  "sentiment": "positive",
  "needs_response": true,
  "response": "Thanks so much for the love! We’re really glad you’re enjoying the new app design. If you have any ideas or features you’d like to see next, we’re all ears.",
  "support_ticket_description": null
}


In [30]:
#Additional example of output - analyze user moodsm sentiment of the user post
from pydoc import text


class UserPost(BaseModel):
    message: str

def make_post(output_class):
    completion = client.responses.parse(
        model="gpt-5.1",
        input=[
            {"role": "system", "content": """
                You are a customer of Tech Corp (@techcorp), a company
                that provides an app and a website. Create a small 
                microblog-style post to them that sends some kind of 
                feedback, positive or negative.
            """},
            {"role": "user", "content": "Please write a post."},
        ],
        text_format=output_class,
    )
    return completion.output_parsed

new_post = make_post(UserPost)
new_post

UserPost(message='@techcorp Really liking the new app layout—much cleaner and faster. One issue though: dark mode doesn’t stay enabled after closing the app on Android. Could you look into making the theme setting persist across sessions?')

In [32]:
analyze_mention(new_post.message)

Mention(product='app', sentiment='positive', needs_response=True, response='Thanks so much for the feedback—we’re glad you’re enjoying the new layout! The issue you’re seeing with dark mode not staying enabled on Android isn’t expected behavior. \n\nA couple quick questions to help us troubleshoot:\n- Which Android version and device model are you using?\n- What app version do you currently have installed?\n\nIn the meantime, you can try clearing the app cache and then setting dark mode again to see if it helps. We’ll also share your request about making the theme setting persist across sessions with our product team so it’s handled more robustly. Thanks for helping us improve the app!', support_ticket_description='User reports that in the Android app, dark mode does not persist after closing and reopening the app. They would like the theme setting (dark mode) to be persistent across sessions. Please investigate settings persistence for appearance/theme on latest Android build and veri

In [33]:
## Programming with structured outputs

In [34]:
from helper import print_mention

# Loop through posts that tagged us and store the results in a list
rows = []
for mention in mentions:
    # Call the LLM to get a Mention object we can program with
    processed_mention = analyze_mention(mention)

    # Print out some information
    print_mention(processed_mention, mention)
    
    # Convert our processed data to a dictionary
    # using Pydantic tools
    processed_dict = processed_mention.model_dump()
    
    # Store the original message in the dataframe row
    processed_dict['mention'] = mention
    rows.append(processed_dict)
    
    print("") # Add separator to make it easier to read

Responding to positive app feedback
  User: @techcorp your app is amazing! The new design is perfect
  Response: Thank you so much for the love! We’re really glad you’re enjoying the new design. If you have any ideas or features you’d love to see next, we’re all ears 🙌

Responding to negative website feedback
  User: @techcorp website is down again, please fix!
  Response: Thanks for letting us know and sorry for the trouble! Our team is looking into the website issue right now. If you can, please try clearing your cache or using a different browser while we work on a fix. We’ll share an update as soon as everything is back up and running.
  Adding support ticket: User reports that the TechCorp website is down again. Investigate current site uptime, recent outages, and any recurring availability issues. Check monitoring logs and incident history; prioritize restoring access and improving stability.

Not responding to negative not_applicable post
  User: hey @techcorp you're so evil

